In [6]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 8, Finished, Available, Finished)

In [7]:
df = spark.read.format("csv").option("header","true").load("Files/Bronze_Layer/*.csv")
# df now is a Spark DataFrame containing CSV data from "Files/Bronze_Layer/2019.csv".
display(df)

StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 16354211-045c-480f-8ef7-d3f91ceda011)

In [8]:
from pyspark.sql.types import *
from pyspark.sql.functions import * 

StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 10, Finished, Available, Finished)

In [9]:
orderSchema = StructType ([
StructField("SalesOrderNumber", StringType()),
StructField("SalesOrderLineNumber", IntegerType()),
StructField("OrderDate" , DateType()),
StructField("CustomerName" , StringType()),
StructField("Email", StringType()),
StructField("Item", StringType()),
StructField("Quantity", IntegerType()),
StructField("Unitprice" , FloatType()),
StructField("Tax", FloatType()),
])

df = spark.read.format("csv").option("header","true").schema(orderSchema).load("Files/Bronze_Layer/*.csv")

StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 11, Finished, Available, Finished)

In [10]:
display(df)

StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2651cd86-4f61-4661-a205-6406dbe07fad)

In [11]:
df = df.withColumn('FileName', input_file_name()).withColumn('IsFlagged', when(col("OrderDate") < '2019-08-01', True).otherwise(False))\
.withColumn('CreatedTs', current_timestamp()).withColumn('ModifedTs', current_timestamp())

StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 13, Finished, Available, Finished)

In [12]:
display(df)


StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b9d25fa5-7c68-41fb-b00b-bf660f1ca977)

In [13]:
df  = df.withColumn('CustomerName', when((col('CustomerName').isNull()) | (col('CustomerName') == ''), lit('UnKnown')).otherwise(col('CustomerName')))


StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 15, Finished, Available, Finished)

In [14]:
display(df)

StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 09f044c0-b51f-40fd-952c-dfa779e47a84)

In [15]:
df = spark.read.table('sales_silver')

StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 17, Finished, Available, Finished)

In [16]:
from delta.tables import * 
table_path = 'Tables/dbo/sales_silver'
if DeltaTable.isDeltaTable(spark,table_path):
    delta = DeltaTable.forPath(spark,table_path)
    delta.alias('silver').merge(
        df.alias('update'),
        'silver.SalesOrderNumber = update.SalesOrderNumber'
    ).whenNotMatchedInsertAll().execute()
else:
    df.write.format('delta').save(table_path)
    

StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 18, Finished, Available, Finished)

In [17]:
from delta.tables import * 
from pyspark.sql.types import * 
from pyspark.sql.functions import *
DeltaTable.createIfNotExists(spark)\
.tableName('dbo.dimdate_gold')\
.addColumn('OrderDate', DateType()).addColumn('Day', IntegerType()).addColumn('Month', IntegerType())\
.addColumn('Year' , IntegerType()).addColumn('mmyyyy', StringType()).execute()


StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 19, Finished, Available, Finished)

In [18]:
dimdate_gold = df.drop_duplicates(['OrderDate']).select(col('OrderDate'), month(col('OrderDate')).alias('Month'), dayofmonth(col('OrderDate')).alias('Day'), year(col('OrderDate')).alias('Year'), date_format(col('OrderDate'), 'MMM-yyyy').alias('mmmyyyy'))


StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 20, Finished, Available, Finished)

In [19]:
display(dimdate_gold)


StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 81d15b70-1084-4407-8337-8d08a44bfe70)

In [20]:
type(dimdate_gold)


StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 22, Finished, Available, Finished)

pyspark.sql.dataframe.DataFrame

In [21]:
dimdate_gold = dimdate_gold.withColumnRenamed("mmmyyyy", "mmyyyy")

table_path = 'Tables/dbo/dimdate_gold'
if DeltaTable.isDeltaTable(spark, table_path):
    delta = DeltaTable.forPath(spark, table_path)
    delta.alias('gold').merge(
        dimdate_gold.alias('update'),
        'gold.OrderDate = update.OrderDate'
    ).whenNotMatchedInsertAll().execute()
else:
    dimdate_gold.write.format('delta').mode('overwrite').save(table_path)





StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 23, Finished, Available, Finished)

In [22]:
display(df)

StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 24, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 79340861-d0f3-4595-82af-ee7df44874ec)

In [23]:
df_gold_cusomter = df.dropDuplicates(['CustomerName', 'Email']).select('CustomerName', 'Email')\
    .withColumn('First_Name', split(col('CustomerName'), ' ')[0])\
    .withColumn('Last_Name', split(col('CustomerName'), ' ')[1])
    


StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 25, Finished, Available, Finished)

In [24]:
table_path = 'dbo.dimcustomergold'

if spark.catalog.tableExists(table_path):
    exist_table = spark.read.table(table_path)
    max_id = exist_table.select(coalesce(max(col('CustomerId')), lit(0)).alias('max_id')).first()['max_id']

    df_new_customer = df_gold_cusomter.join(exist_table, ['CustomerName', 'Email'], 'left_anti')\
        .withColumn('CustomerId', monotonically_increasing_id() + lit(max_id) + lit(1))

    df_new_customer.write.mode('append').format('delta').saveAsTable(table_path)

else:
    df_gold_cusomter.withColumn('CustomerId', monotonically_increasing_id() + lit(1))\
        .write.format('delta').saveAsTable(table_path)


StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 26, Finished, Available, Finished)

StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 38, Finished, Available, Finished)

In [39]:
df_product = df.withColumn('ItemName', split(col('Item'), ', ').getItem(0))\
    .withColumn('ItemInfo', split(col('Item'), ', ').getItem(1))\
    .select('ItemName', 'ItemInfo')

table_path = 'dbo.dimproductgold'

if spark.catalog.tableExists(table_path):
    exist_table = spark.read.table(table_path)
    max_id = exist_table.select(coalesce(max(col('ItemId')), lit(0))).first()[0]

    df_new_product = df_product.join(exist_table, ['ItemName', 'ItemInfo'], 'left_anti')\
        .withColumn('ItemId', monotonically_increasing_id() + lit(max_id) + lit(1))

    df_new_product.write.mode('append').format('delta').saveAsTable(table_path)

else:
    df_product.withColumn('ItemId', monotonically_increasing_id() + lit(1))\
        .write.format('delta').saveAsTable(table_path)


StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 41, Finished, Available, Finished)

In [41]:
from delta.tables import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 43, Finished, Available, Finished)

In [43]:
dim_cus = spark.read.table('dbo.dimcustomergold')
dim_pro = spark.read.table('dbo.dimproductgold')

df_prep = df.withColumn('ItemName', split(col('Item'), ', ').getItem(0))\
    .withColumn('ItemInfo', split(col('Item'), ', ').getItem(1))

df_fact = df_prep.join(dim_cus,['CustomerName', 'Email']).join(dim_pro,['ItemName' , 'ItemInfo'], 'left')\
.select('CustomerId', 'ItemId', 'OrderDate', 'Quantity' , 'UnitPrice', 'Tax')


StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 45, Finished, Available, Finished)

In [46]:
table_path = 'Tables/dbo/FactSales_gold'

if DeltaTable.isDeltaTable(spark, table_path):
    delta = DeltaTable.forPath(spark, table_path)
    delta.alias('gold').merge(
        df_fact.alias('update'),
        'gold.OrderDate = update.OrderDate AND gold.CustomerID = update.CustomerID AND gold.ItemID = update.ItemID'
    ).whenNotMatchedInsertAll().execute()
else:
    df_fact.write.format('delta').mode('overwrite').save(table_path)


StatementMeta(, 730dec7e-72a9-4507-aa29-615385638c75, 48, Finished, Available, Finished)